# Test ModSec Analyzer

Notebook pour tester le module `modsec_analyzer`.

In [1]:
# Ajouter le package au path
import sys
sys.path.insert(0, "packages/modsec_analyzer/src")

# Autoreload pour recharger les modules modifiés
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
from modsec_analyzer.parsing import parse_file
from modsec_analyzer.domain import (
    Rule, SecRule, SecAction, Directive,
    Variable, Operator, Action, Argument,
    RuleSet
)
from collections import Counter

In [ ]:
# Chemin vers un fichier de configuration CRS
# Modifier ce chemin selon votre installation
CRS_PATH = "config/modsecurity/setup.conf"
# ou un fichier de règles spécifique
# CRS_PATH = "/path/to/coreruleset/rules/REQUEST-942-APPLICATION-ATTACK-SQLI.conf"

## Test parsing avec résolution des includes

In [4]:
# Parser avec résolution des includes (peut prendre quelques secondes)
ruleset_full = parse_file(CRS_PATH, resolve_includes=True)

print(f"Nombre total de rules: {len(ruleset_full.rules)}")

FileNotFoundError: [Errno 2] No such file or directory: '/home/lucas/reforge/WAF-GUARD/experiments/experiments/config/modsecurity/setup.conf'

In [ ]:
# Fichiers sources uniques
unique_files = set(rule.file_path for rule in ruleset_full.rules)
print(f"\nFichiers parsés ({len(unique_files)}):")
for f in sorted(unique_files):
    print(f"  {f}")


Fichiers parsés (28):
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs-setup.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-901-INITIALIZATION.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-905-COMMON-EXCEPTIONS.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-911-METHOD-ENFORCEMENT.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-913-SCANNER-DETECTION.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-920-PROTOCOL-ENFORCEMENT.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-921-PROTOCOL-ATTACK.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-922-MULTIPART-ATTACK.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-930-APPLICATION-ATTACK-LFI.conf
  /home/lucas/reforge/WAF-GUARD/

In [ ]:
print(ruleset_full.filter(filename="crs_rules"))

RuleSet (715 rules from 26 file(s))
  Types:
    SecRule: 677
    SecMarker: 30
    SecAction: 7
    SecComponentSignature: 1

  Tags:
    attack:
      rce: 90
      protocol: 74
      sqli: 60
      xss: 33
      disclosure: 33
      injection-php: 20
      injection-generic: 7
      multipart-header: 6
      lfi: 5
      rfi: 5
      generic: 3
      ssrf: 3
      fixation: 3
      reputation-scanner: 1
      deprecated-header: 1
      ssti: 1
      injection-java: 1
      (no tag): 347
    application:
      multi: 308
      (no tag): 376
    language:
      multi: 218
      php: 52
      shell: 33
      java: 15
      javascript: 4
      ruby: 3
      aspnet: 2
      ldap: 1
      powershell: 1
      perl: 1
      (no tag): 355
    platform:
      multi: 254
      unix: 27
      windows: 13
      internet-explorer: 13
      iis: 6
      nodejs: 5
      apache: 3
      tomcat: 1
      msaccess: 1
      oracle: 1
      db2: 1
      emc: 1
      firebird: 1
      frontbase: 1
      h

In [ ]:
counter=0
for i in range(1, 6):
    rules = ruleset_full.filter(phase=i)
    print(f"\nRègles pour la phase {i} : {len(rules)}")
    counter += len(rules)
print(f"\nTotal des règles phasées: {counter}")


Règles pour la phase 1 : 173

Règles pour la phase 2 : 280

Règles pour la phase 3 : 43

Règles pour la phase 4 : 109

Règles pour la phase 5 : 13

Total des règles phasées: 618


In [ ]:
rules=ruleset_full.filter(pl=1)
print(f"\nRègles pour le niveau de paranoïa 1 : {len(rules)}")
rules=ruleset_full.filter(pl=2)
print(f"\nRègles pour le niveau de paranoïa 2 : {len(rules)}")
rules=ruleset_full.filter(pl=3)
print(f"\nRègles pour le niveau de paranoïa 3 : {len(rules)}")
rules=ruleset_full.filter(pl=4)
print(f"\nRègles pour le niveau de paranoïa 4 : {len(rules)}")


Règles pour le niveau de paranoïa 1 : 604

Règles pour le niveau de paranoïa 2 : 692

Règles pour le niveau de paranoïa 3 : 721

Règles pour le niveau de paranoïa 4 : 730


In [ ]:
rules=ruleset_full.filter(action="chain")
print(rules)

RuleSet (70 rules from 13 file(s))
  Types:
    SecRule: 70

  Tags:
    attack:
      protocol: 28
      rce: 8
      sqli: 4
      generic: 2
      rfi: 2
      fixation: 2
      multipart-header: 1
      xss: 1
      disclosure: 1
      (no tag): 21
    application:
      multi: 48
      (no tag): 22
    language:
      multi: 45
      java: 2
      aspnet: 1
      shell: 1
      (no tag): 21
    platform:
      multi: 43
      apache: 2
      windows: 2
      unix: 1
      tomcat: 1
      iis: 1
      (no tag): 21


## Analyse visuelle — Impact du Paranoia Level

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# CRS rules only
crs = ruleset_full.filter(filename="crs_rules")

# Cumulative rulesets per PL (filter(pl=N) returns rules with PL <= N)
pl_rulesets = {pl: crs.filter(pl=pl) for pl in range(1, 5)}

# Severity order for consistent plotting
SEVERITY_ORDER = ["CRITICAL", "ERROR", "WARNING", "NOTICE", "(none)"]
SEVERITY_COLORS = {
    "CRITICAL": "#d32f2f",
    "ERROR": "#f57c00",
    "WARNING": "#fbc02d",
    "NOTICE": "#64b5f6",
    "(none)": "#bdbdbd",
}

In [ ]:
# Plot 4 — Severity Distribution globale
sev_dist = crs.severity_distribution
sev_labels = [s for s in SEVERITY_ORDER if sev_dist.get(s, 0) > 0]
sev_values = [sev_dist[s] for s in sev_labels]
sev_colors = [SEVERITY_COLORS[s] for s in sev_labels]

fig, ax = plt.subplots(figsize=(8, 3))
bars = ax.barh(sev_labels, sev_values, color=sev_colors)
ax.bar_label(bars, fmt="%d", padding=4)
ax.set_xlabel("Nombre d'action rules")
ax.set_title("Distribution des sévérités — CRS Rules")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Plot 1 — Composition du Ruleset par Paranoia Level (donut charts)
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i, pl in enumerate(range(1, 5)):
    ax = axes[i]
    sev = pl_rulesets[pl].severity_distribution
    total = sum(sev.values())

    labels = [s for s in SEVERITY_ORDER if sev.get(s, 0) > 0]
    sizes = [sev[s] for s in labels]
    colors = [SEVERITY_COLORS[s] for s in labels]

    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=None,
        colors=colors,
        autopct=lambda pct: f"{int(round(pct * total / 100))}",
        pctdistance=0.75,
        startangle=90,
        wedgeprops={"width": 0.45, "edgecolor": "white", "linewidth": 1.5},
    )
    for t in autotexts:
        t.set_fontsize(8)

    # Total au centre
    ax.text(0, 0, str(total), ha="center", va="center", fontsize=16, fontweight="bold")
    ax.set_title(f"PL{pl}", fontsize=13, fontweight="bold")

# Légende commune
fig.legend(
    [plt.Rectangle((0, 0), 1, 1, fc=SEVERITY_COLORS[s]) for s in SEVERITY_ORDER if crs.severity_distribution.get(s, 0) > 0],
    [s for s in SEVERITY_ORDER if crs.severity_distribution.get(s, 0) > 0],
    loc="center right",
    title="Severity",
    bbox_to_anchor=(1.02, 0.5),
)
fig.suptitle("Composition du Ruleset par Paranoia Level", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 2 — Heatmap Attack Coverage × PL (cumulative)
# Build matrix: rows = attack sub-types, columns = PL levels
attack_types = [
    t for t, _ in crs.tag_distribution("attack").most_common()
    if t != "(untagged)"
]

heatmap_data = {}
for pl in range(1, 5):
    dist = pl_rulesets[pl].tag_distribution("attack")
    heatmap_data[f"PL{pl}"] = {t: dist.get(t, 0) for t in attack_types}

df_heatmap = pd.DataFrame(heatmap_data, index=attack_types)

fig, ax = plt.subplots(figsize=(6, max(5, len(attack_types) * 0.4)))
sns.heatmap(
    df_heatmap,
    annot=True,
    fmt="d",
    cmap="YlOrRd",
    linewidths=0.5,
    ax=ax,
    cbar_kws={"label": "Nombre de rules"},
)
ax.set_title("Couverture par type d'attaque × Paranoia Level")
ax.set_ylabel("Type d'attaque")
ax.set_xlabel("Paranoia Level (cumulatif)")
plt.tight_layout()
plt.show()

In [ ]:
# Plot 3 — Heatmap normalisée (% du PL4)
# Each row normalized to 100% = PL4 count for that attack type
df_normalized = df_heatmap.div(df_heatmap["PL4"], axis=0) * 100
# Replace NaN (0/0 for types with 0 rules at PL4) with 0
df_normalized = df_normalized.fillna(0)

fig, ax = plt.subplots(figsize=(6, max(5, len(attack_types) * 0.4)))
sns.heatmap(
    df_normalized,
    annot=True,
    fmt=".0f",
    cmap="RdYlGn",
    vmin=0,
    vmax=100,
    linewidths=0.5,
    ax=ax,
    cbar_kws={"label": "% de couverture (PL4 = 100%)"},
)
ax.set_title("% de couverture atteint par Paranoia Level (PL4 = 100%)")
ax.set_ylabel("Type d'attaque")
ax.set_xlabel("Paranoia Level (cumulatif)")
plt.tight_layout()
plt.show()

---

## Étude de cas : Faux positif SQLi sur un champ de recherche

**Scénario** : Une application e-commerce expose un endpoint de recherche `/api/search?q=...`.
Un utilisateur légitime effectue la requête :

```
GET /api/search?q=SELECT+model+FROM+our+catalog+WHERE+price+UNION+ALL
```

Ce type de requête — parfaitement légitime dans un contexte métier (documentation SQL, outil d'administration, CMS technique) — déclenche plusieurs règles CRS de la catégorie **REQUEST-942 (SQL Injection)**.

**Objectif** : Identifier les règles responsables, calculer le score d'anomalie résultant, et choisir la stratégie d'exclusion appropriée parmi les 4 approches CRS.

**Paramètres** : Paranoia Level 2, seuil d'anomalie inbound par défaut (5).

In [ ]:
# === Paramètres de l'étude de cas ===
PARANOIA_LEVEL = 2
INBOUND_ANOMALY_THRESHOLD = 5  # Seuil par défaut CRS

# Correspondance severity → score d'anomalie (CRS standard)
SEVERITY_SCORE = {
    "CRITICAL": 5,
    "ERROR": 4,
    "WARNING": 3,
    "NOTICE": 2,
}

# Filtrer les règles CRS au PL choisi
crs_pl = ruleset_full.filter(filename="crs_rules", pl=PARANOIA_LEVEL)
print(f"Règles CRS actives à PL{PARANOIA_LEVEL}: {len(crs_pl)}")
print(f"Seuil d'anomalie inbound: {INBOUND_ANOMALY_THRESHOLD}")

# Distribution des sévérités à ce PL
sev = crs_pl.severity_distribution
print(f"\nDistribution des sévérités à PL{PARANOIA_LEVEL}:")
for s in SEVERITY_ORDER:
    count = sev.get(s, 0)
    if count > 0:
        score = SEVERITY_SCORE.get(s, 0)
        print(f"  {s:<10}: {count:>3} règles (score unitaire: +{score})")

### Etape 1 — Identifier les règles SQLi actives au PL choisi

In [ ]:
# Règles SQLi (fichier 942) actives à PL2
sqli_rules = crs_pl.filter(filename="942")
print(f"=== REQUEST-942: SQL Injection (PL{PARANOIA_LEVEL}) ===\n")
print(sqli_rules)

In [ ]:
# Identifier les règles qui inspectent ARGS (paramètres de requête)
# Ce sont celles susceptibles de matcher notre ?q=SELECT+model+FROM...
args_sqli_rules = []
for rule in sqli_rules:
    if isinstance(rule, SecRule):
        var_names = [v.variable for v in rule.variables]
        if any(v in ("ARGS", "ARGS_GET", "ARGS_NAMES", "REQUEST_URI",
                      "REQUEST_LINE", "QUERY_STRING", "REQUEST_BODY")
               for v in var_names):
            args_sqli_rules.append(rule)

print(f"Règles SQLi inspectant les paramètres de requête: {len(args_sqli_rules)}")
print(f"(sur {len(sqli_rules)} règles SQLi totales au PL{PARANOIA_LEVEL})\n")

# Afficher les règles potentiellement déclenchées
for rule in args_sqli_rules[:15]:
    rule_id = rule.get_action("id")
    msg = rule.get_action("msg")
    severity = rule.get_action("severity")
    score = SEVERITY_SCORE.get(severity, 0)
    variables = ", ".join(
        v.variable + (f":{v.variable_part}" if v.variable_part else "")
        for v in rule.variables
    )
    op_arg = rule.operator.operator_argument
    op_display = op_arg[:70] + "..." if len(op_arg) > 70 else op_arg

    print(f"  [{rule_id}] {msg}")
    print(f"    Severity: {severity} (+{score})  |  Variables: {variables}")
    print(f"    Operator: {rule.operator.operator} \"{op_display}\"")
    print()

if len(args_sqli_rules) > 15:
    print(f"  ... et {len(args_sqli_rules) - 15} autres règles")

### Etape 2 — Calculer le score d'anomalie potentiel

Le CRS fonctionne en **anomaly scoring** : chaque règle déclenchée incrémente un compteur (`tx.anomaly_score`) selon sa sévérité. Si le score cumulé atteint le seuil (`tx.inbound_anomaly_score_threshold`, défaut: 5), la requête est bloquée.

| Severity | Score |
|----------|-------|
| CRITICAL | +5    |
| ERROR    | +4    |
| WARNING  | +3    |
| NOTICE   | +2    |

In [ ]:
# Distribution des sévérités parmi les règles SQLi ciblant ARGS
sqli_args_ruleset = RuleSet(rules=args_sqli_rules)
sev_dist = sqli_args_ruleset.severity_distribution

print(f"=== Distribution des sévérités (SQLi sur ARGS, PL{PARANOIA_LEVEL}) ===\n")
for severity in SEVERITY_ORDER:
    count = sev_dist.get(severity, 0)
    if count > 0:
        score = SEVERITY_SCORE.get(severity, 0)
        print(f"  {severity:<10}: {count:>3} règles  x  +{score}  =  {count * score:>4} points potentiels")

total_potential = sum(sev_dist.get(s, 0) * SEVERITY_SCORE.get(s, 0) for s in SEVERITY_ORDER)
print(f"\n  Score max si toutes déclenchées: {total_potential} points")
print(f"  Seuil de blocage:               {INBOUND_ANOMALY_THRESHOLD} points")
print(f"  --> Il suffit d'1 seule règle CRITICAL pour bloquer la requête")

In [ ]:
# Simulation : accumulation du score d'anomalie règle par règle
print(f"=== Simulation : accumulation du score (pire cas, PL{PARANOIA_LEVEL}) ===\n")

total_score = 0
triggered_rules = []

for rule in args_sqli_rules:
    severity = rule.get_action("severity")
    score = SEVERITY_SCORE.get(severity, 0)
    if score > 0:
        total_score += score
        triggered_rules.append({
            "id": rule.get_action("id"),
            "msg": rule.get_action("msg"),
            "severity": severity,
            "score": score,
            "cumulative": total_score,
        })

print(f"{'ID':<12} {'Severity':<10} {'Score':>5} {'Cumul':>6}  Message")
print("-" * 105)
for r in triggered_rules:
    just_crossed = (r["cumulative"] >= INBOUND_ANOMALY_THRESHOLD
                    and (r["cumulative"] - r["score"]) < INBOUND_ANOMALY_THRESHOLD)
    marker = " <-- BLOQUE" if just_crossed else ""
    over = " !!" if r["cumulative"] >= INBOUND_ANOMALY_THRESHOLD else ""
    msg_short = (r["msg"][:50] + "...") if r["msg"] and len(r["msg"]) > 50 else (r["msg"] or "")
    print(f"{r['id']:<12} {r['severity']:<10} +{r['score']:<4} {r['cumulative']:>5}{over}  {msg_short}{marker}")

print(f"\nScore total potentiel: {total_score}")
print(f"Seuil de blocage: {INBOUND_ANOMALY_THRESHOLD}")
print(f"Dépassement: {'OUI' if total_score >= INBOUND_ANOMALY_THRESHOLD else 'NON'} ({total_score - INBOUND_ANOMALY_THRESHOLD:+d} points)")

In [ ]:
# Visualisation : accumulation du score d'anomalie
if triggered_rules:
    fig, ax = plt.subplots(figsize=(14, 5))

    ids = [r["id"] for r in triggered_rules]
    cumulative = [r["cumulative"] for r in triggered_rules]
    scores = [r["score"] for r in triggered_rules]
    colors = [SEVERITY_COLORS.get(r["severity"], "#999") for r in triggered_rules]

    ax.bar(range(len(ids)), scores, color=colors, alpha=0.7, label="Score individuel")
    ax.plot(range(len(ids)), cumulative, "k-o", markersize=4, linewidth=1.5, label="Score cumulé")
    ax.axhline(
        y=INBOUND_ANOMALY_THRESHOLD, color="red", linestyle="--",
        linewidth=2, label=f"Seuil blocage ({INBOUND_ANOMALY_THRESHOLD})"
    )

    # Zone de blocage
    ax.axhspan(INBOUND_ANOMALY_THRESHOLD, max(cumulative) * 1.1,
               alpha=0.08, color="red", label="Zone de blocage")

    ax.set_xticks(range(len(ids)))
    ax.set_xticklabels(ids, rotation=90, fontsize=7)
    ax.set_xlabel("Rule ID")
    ax.set_ylabel("Score d'anomalie")
    ax.set_title(f"Accumulation du score d'anomalie — SQLi sur ARGS (PL{PARANOIA_LEVEL})")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

### Etape 3 — Analyse par tag : score atteignable par catégorie d'attaque

Utilisons `tag_distribution` pour comprendre la surface d'attaque par catégorie et le score d'anomalie maximum atteignable pour chacune.

In [ ]:
# Score d'anomalie max atteignable par catégorie d'attaque (PL2)
# Pour chaque tag attack-*, on calcule le score total si toutes les règles matchent

attack_dist = crs_pl.tag_distribution("attack")
attack_types_sorted = [t for t, _ in attack_dist.most_common() if t != "(untagged)"]

# Pour chaque type d'attaque, calculer le score max
attack_scores = {}
for attack_type in attack_types_sorted:
    tag_full = f"attack-{attack_type}"
    # Trouver les règles avec ce tag
    matching_rules = [
        r for r in crs_pl.action_rules
        if any(t.startswith(tag_full) for t in r.tags)
    ]
    total_score = sum(
        SEVERITY_SCORE.get(r.get_action("severity"), 0)
        for r in matching_rules
    )
    attack_scores[attack_type] = {
        "count": len(matching_rules),
        "score": total_score,
        "severities": Counter(r.get_action("severity") or "(none)" for r in matching_rules),
    }

print(f"=== Score d'anomalie max par type d'attaque (PL{PARANOIA_LEVEL}) ===\n")
print(f"{'Type attaque':<25} {'Rules':>6} {'Score max':>10}  Détail sévérités")
print("-" * 85)
for attack_type in attack_types_sorted:
    info = attack_scores[attack_type]
    sev_detail = ", ".join(
        f"{s}:{c}" for s, c in info["severities"].most_common() if s != "(none)"
    )
    bar = "#" * (info["score"] // 5)
    print(f"  {attack_type:<23} {info['count']:>6} {info['score']:>10}  {bar}  {sev_detail}")

# Highlight SQLi
sqli_info = attack_scores.get("sqli", {})
print(f"\n--> SQLi : {sqli_info.get('count', 0)} règles, score max {sqli_info.get('score', 0)} points")
print(f"    Avec un seuil de {INBOUND_ANOMALY_THRESHOLD}, il suffit que quelques règles matchent pour bloquer")

In [ ]:
# Visualisation : score max atteignable par type d'attaque
fig, ax = plt.subplots(figsize=(10, 6))

types = list(reversed(attack_types_sorted))
scores_list = [attack_scores[t]["score"] for t in types]
counts = [attack_scores[t]["count"] for t in types]

# Color bars: highlight sqli
bar_colors = ["#d32f2f" if t == "sqli" else "#1976d2" for t in types]

bars = ax.barh(types, scores_list, color=bar_colors, alpha=0.8)

# Annoter avec le nombre de règles et le score
for i, (bar, score, count) in enumerate(zip(bars, scores_list, counts)):
    if score > 0:
        ax.text(score + 2, bar.get_y() + bar.get_height() / 2,
                f"{score} pts ({count} rules)", va="center", fontsize=8)

ax.axvline(x=INBOUND_ANOMALY_THRESHOLD, color="red", linestyle="--",
           linewidth=1.5, label=f"Seuil blocage ({INBOUND_ANOMALY_THRESHOLD})")
ax.set_xlabel("Score d'anomalie max atteignable")
ax.set_title(f"Score d'anomalie max par type d'attaque — CRS PL{PARANOIA_LEVEL}\n(SQLi en rouge)")
ax.legend()
plt.tight_layout()
plt.show()

### Etape 4 — Stratégies d'exclusion du faux positif

Le CRS propose **4 approches** pour gérer les faux positifs, selon 2 axes :

|  | **Suppression de règle** | **Mise à jour de cible** |
|--|--------------------------|--------------------------|
| **Configure-time** | `SecRuleRemoveById` / `SecRuleRemoveByTag` | `SecRuleUpdateTargetById` / `SecRuleUpdateTargetByTag` |
| **Runtime** | `ctl:ruleRemoveById` / `ctl:ruleRemoveByTag` | `ctl:ruleRemoveTargetById` / `ctl:ruleRemoveTargetByTag` |

**Axes de décision :**
- **Suppression vs Target Update** : supprimer la règle entièrement, ou seulement exclure la variable problématique ?
- **Configure-time vs Runtime** : appliquer globalement, ou conditionnellement (par URI, IP, etc.) ?

In [ ]:
# === Stratégie 1 — Suppression par ID (configure-time) ===
print("=" * 80)
print("STRATEGIE 1 — SecRuleRemoveById (configure-time)")
print("=" * 80)
print()
print("# Désactiver les règles SQLi/ARGS une par une")
print("# A placer APRES l'inclusion des règles CRS\n")
for rule in args_sqli_rules[:5]:
    rid = rule.get_action("id")
    if rid:
        print(f"SecRuleRemoveById {rid}")
print(f"# ... ({len(args_sqli_rules)} règles au total)")
print()
print("RISQUE : supprime TOUTE la protection SQLi sur TOUTES les variables,")
print("         pas seulement pour le champ 'q', et pour TOUS les endpoints.")

print("\n")
print("=" * 80)
print("STRATEGIE 2 — SecRuleRemoveByTag (configure-time)")
print("=" * 80)
print()

# Tags SQLi présents sur ces règles
sqli_tags = set()
for rule in args_sqli_rules:
    for tag in rule.tags:
        if "sqli" in tag.lower() or "sql" in tag.lower():
            sqli_tags.add(tag)
print(f"Tags SQLi trouvés: {sorted(sqli_tags)}\n")
for tag in sorted(sqli_tags)[:3]:
    print(f'SecRuleRemoveByTag "{tag}"')
print()
print("RISQUE : meme effet que stratégie 1, mais en une seule ligne.")
print("         Supprime toutes les règles avec ce tag.")

In [ ]:
# === Stratégie 3 — SecRuleUpdateTargetById (configure-time) ===
print("=" * 80)
print("STRATEGIE 3 — SecRuleUpdateTargetById (configure-time)  [RECOMMANDEE]")
print("=" * 80)
print()
print("# Exclure seulement le paramètre 'q' des règles SQLi")
print("# Les règles restent actives sur tous les AUTRES paramètres\n")
for rule in args_sqli_rules[:5]:
    rid = rule.get_action("id")
    if rid:
        print(f'SecRuleUpdateTargetById {rid} "!ARGS:q"')
print(f"# ... ({len(args_sqli_rules)} règles au total)")
print()

# Montrer combien de règles ciblent plusieurs variables
multi_var = [r for r in args_sqli_rules if len(r.variables) > 1]
single_var = [r for r in args_sqli_rules if len(r.variables) == 1]
print(f"Variables ciblées par ces règles :")
print(f"  {len(multi_var)} règles ciblent PLUSIEURS variables (protection conservée sur les autres)")
print(f"  {len(single_var)} règles ciblent UNE seule variable")
print()
print("AVANTAGE : exclusion chirurgicale, couverture SQLi intacte sur les autres params.")
print("LIMITE   : s'applique globalement à tous les endpoints.")

In [ ]:
# === Stratégie 4 — Runtime conditionnel (ctl:) ===
print("=" * 80)
print("STRATEGIE 4 — ctl:ruleRemoveTargetByTag (runtime)  [LA PLUS PRECISE]")
print("=" * 80)
print()
print("# Exclure ARGS:q seulement pour l'endpoint /api/search")
print("# La protection SQLi reste intacte partout ailleurs\n")

# Identifier le meilleur tag pour cibler toutes les règles SQLi
attack_sqli_tag = "attack-sqli"
sqli_tag_count = sum(
    1 for r in args_sqli_rules
    if any(t == attack_sqli_tag for t in r.tags)
)

print('SecRule REQUEST_URI "@beginsWith /api/search" \\')
print('    "id:1000,\\')
print('     phase:1,\\')
print('     nolog,\\')
print('     pass,\\')
print(f'     ctl:ruleRemoveTargetByTag={attack_sqli_tag};ARGS:q"')
print()
print(f"Couverture du tag '{attack_sqli_tag}' : {sqli_tag_count}/{len(args_sqli_rules)} règles SQLi/ARGS")
print()
print("AVANTAGE : scope minimal — seul ARGS:q sur /api/search est exclu.")
print("           Protection SQLi intacte sur tous les autres endpoints ET paramètres.")
print("           Une seule directive grâce au ciblage par tag.")

### Etape 5 — Comparaison d'impact sur la couverture de sécurité

In [ ]:
# === Comparaison d'impact des stratégies ===
print(f"=== Impact sur la couverture de sécurité (PL{PARANOIA_LEVEL}) ===\n")

total_crs = len(crs_pl)
total_sqli_args = len(args_sqli_rules)

# Score de protection perdu par stratégie
score_lost_remove = sum(
    SEVERITY_SCORE.get(r.get_action("severity"), 0) for r in args_sqli_rules
)

# Stratégie 1 & 2: Remove rules entirely
remaining_remove = total_crs - total_sqli_args
coverage_remove = remaining_remove / total_crs * 100

print(f"{'Stratégie':<50} {'Rules':>7} {'Couverture':>11} {'Scope'}")
print("-" * 100)
print(f"{'Baseline (PL' + str(PARANOIA_LEVEL) + ')':<50} {total_crs:>7} {'100.0%':>11} {'Global'}")
print(f"{'1. SecRuleRemoveById (toutes SQLi/ARGS)':<50} {remaining_remove:>7} {coverage_remove:>10.1f}% {'Global — protection SQLi supprimée'}")
print(f"{'2. SecRuleRemoveByTag (idem par tag)':<50} {remaining_remove:>7} {coverage_remove:>10.1f}% {'Global — protection SQLi supprimée'}")
print(f"{'3. SecRuleUpdateTargetById !ARGS:q':<50} {total_crs:>7} {'~100.0%':>11} {'Global — ARGS:q exclu seulement'}")
print(f"{'4. ctl:ruleRemoveTargetByTag (runtime)':<50} {total_crs:>7} {'100.0%':>11} {'/api/search?q= exclu seulement'}")

print(f"\n--- Détail ---")
print(f"Règles SQLi/ARGS supprimées (stratégie 1-2) : {total_sqli_args}")
print(f"Score de protection perdu : {score_lost_remove} points")
print(f"Règles multi-variables conservées (stratégie 3-4) : {len(multi_var)}/{total_sqli_args}")
print(f"\nRecommandation : Stratégie 4 (runtime + target update par tag)")
print(f"  --> Scope minimal : un seul paramètre, un seul endpoint")
print(f"  --> Protection SQLi intacte partout ailleurs")

In [ ]:
# Visualisation : comparaison des stratégies
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Panel gauche : couverture en nombre de règles ---
strategies = [
    f"Baseline\nPL{PARANOIA_LEVEL}",
    "1. RemoveById\n(configure)",
    "2. RemoveByTag\n(configure)",
    "3. UpdateTarget\n(configure)",
    "4. ctl:Target\n(runtime)",
]
rules_active = [total_crs, remaining_remove, remaining_remove, total_crs, total_crs]
rules_colors = ["#4caf50", "#f44336", "#f44336", "#ff9800", "#4caf50"]

bars = axes[0].bar(strategies, rules_active, color=rules_colors, alpha=0.85, edgecolor="white")
axes[0].bar_label(bars, fmt="%d", padding=4)
axes[0].set_ylabel("Nombre de règles actives")
axes[0].set_title("Règles actives après exclusion")
axes[0].set_ylim(0, total_crs * 1.15)

# --- Panel droit : scope d'exclusion ---
scope_labels = ["Paramètres\nexclus", "Endpoints\naffectés", "Types attaque\nexclus"]
# Stratégie 1-2 : tous les params, tous les endpoints, SQLi complet
# Stratégie 3 : ARGS:q seulement, tous les endpoints
# Stratégie 4 : ARGS:q seulement, /api/search seulement

scope_data = {
    "1-2. Remove": [100, 100, 100],  # tout exclu
    "3. UpdateTarget": [5, 100, 0],   # 1 param, global, pas de type supprimé
    "4. ctl:Target": [5, 5, 0],       # 1 param, 1 endpoint, pas de type supprimé
}

x = np.arange(len(scope_labels))
width = 0.25
for i, (label, values) in enumerate(scope_data.items()):
    color = ["#f44336", "#ff9800", "#4caf50"][i]
    axes[1].bar(x + i * width, values, width, label=label, color=color, alpha=0.85)

axes[1].set_ylabel("% de surface affectée par l'exclusion")
axes[1].set_title("Surface d'impact de l'exclusion (plus bas = mieux)")
axes[1].set_xticks(x + width)
axes[1].set_xticklabels(scope_labels)
axes[1].legend()
axes[1].set_ylim(0, 130)

plt.tight_layout()
plt.show()

### Etape 6 — Approche par tag : analyse détaillée et exclusion groupée

In [ ]:
# Analyse des tags présents sur les règles SQLi/ARGS pour une exclusion groupée
tag_counter = Counter()
for rule in args_sqli_rules:
    tag_counter.update(rule.tags)

# Filtrer les tags pertinents (exclure les metadata CRS génériques)
print("=== Tags les plus fréquents sur les règles SQLi/ARGS ===\n")
print(f"{'Tag':<55} {'Count':>5} {'%':>7}")
print("-" * 70)
for tag, count in tag_counter.most_common(20):
    pct = count / len(args_sqli_rules) * 100
    print(f"  {tag:<53} {count:>5} ({pct:5.1f}%)")

# Vérifier si attack-sqli couvre toutes les règles
attack_sqli_count = tag_counter.get("attack-sqli", 0)
print(f"\n--> Le tag 'attack-sqli' couvre {attack_sqli_count}/{len(args_sqli_rules)} règles "
      f"({attack_sqli_count/len(args_sqli_rules)*100:.0f}%)")

# Règles non couvertes par attack-sqli
uncovered = [r for r in args_sqli_rules if "attack-sqli" not in r.tags]
if uncovered:
    print(f"\nRègles NON couvertes par 'attack-sqli' ({len(uncovered)}) :")
    for r in uncovered:
        rid = r.get_action("id")
        msg = r.get_action("msg")
        tags = [t for t in r.tags if t.startswith("attack-")]
        print(f"  [{rid}] {msg}")
        print(f"    Tags attack: {tags}")
else:
    print("\nToutes les règles sont couvertes par 'attack-sqli'.")

### Conclusion

**Diagnostic** : Le paramètre `q` sur `/api/search` déclenche un faux positif massif sur les règles SQLi (REQUEST-942). Le score d'anomalie dépasse largement le seuil de blocage dès la première règle CRITICAL.

**Recommandation** : **Stratégie 4** — exclusion runtime par tag avec mise à jour de cible :

```apache
SecRule REQUEST_URI "@beginsWith /api/search" \
    "id:1000,\
     phase:1,\
     nolog,\
     pass,\
     ctl:ruleRemoveTargetByTag=attack-sqli;ARGS:q"
```

**Pourquoi cette approche ?**
- **Scope minimal** : seul `ARGS:q` est exclu, et uniquement sur `/api/search`
- **Protection conservée** : les règles SQLi restent actives sur tous les autres paramètres et endpoints
- **Maintenable** : le tag `attack-sqli` couvre automatiquement les futures mises à jour du CRS
- **Une seule directive** : pas besoin de lister chaque ID de règle individuellement